In [1]:
import requests

def make_request(query):
    base_url = "http://api.conceptnet.io"
    if not query.startswith(base_url):
        query = f"{base_url}{query}"
    return requests.get(query).json()

def read_edges(obj):
    for edge in obj['edges']:
        if edge['start']['language'] == 'en' and edge['end']['language'] == 'en':
            yield {
                'edge': (edge['start']['label'], edge['rel']['label'], edge['end']['label']),
                'example': edge['surfaceText']
            }

def get_relations_between_words(word1, word2):
    word1 = word1.replace(' ', '_')
    word2 = word2.replace(' ', '_')
    query = f"/query?node=/c/en/{word1}&other=/c/en/{word2}&language=en"
    obj = make_request(query=query)
    edges = list(read_edges(obj))
    return edges

def get_edges_related_to_word(word):
    word = word.replace(' ', '_')
    query = f"/query?node=/c/en/{word}&language=en"
    obj = make_request(query = query)
    edges = list(read_edges(obj))
    return edges


In [2]:
get_edges_related_to_word('apple')

[{'edge': ('apple', 'RelatedTo', 'fruit'),
  'example': '[[apple]] is related to [[fruit]]'},
 {'edge': ('apple', 'HasProperty', 'red'),
  'example': '[[apple]] can be [[red]]'},
 {'edge': ('apple', 'RelatedTo', 'red'),
  'example': '[[apple]] is related to [[red]]'},
 {'edge': ('apple', 'HasProperty', 'green'),
  'example': '[[apple]] can be [[green]]'},
 {'edge': ('an apple', 'ReceivesAction', 'eaten'),
  'example': '[[an apple]] can be [[eaten]]'},
 {'edge': ('an apple', 'AtLocation', 'apple tree'),
  'example': 'You are likely to find [[an apple]] in [[apple tree]]'},
 {'edge': ('an apple', 'CapableOf', 'fall from a tree'),
  'example': '[[an apple]] can [[fall from a tree]]'},
 {'edge': ('apple', 'UsedFor', 'eating'),
  'example': '[[apple]] is for [[eating]]'},
 {'edge': ('An apple', 'HasA', 'a core'),
  'example': '[[An apple]] has [[a core]]'},
 {'edge': ('apple', 'RelatedTo', 'red fruit'),
  'example': '[[apple]] is related to [[red fruit]]'},
 {'edge': ('apple', 'RelatedTo', 

In [3]:
get_relations_between_words('stepfather', 'father')

[{'edge': ('stepfather', 'DerivedFrom', 'father'), 'example': None},
 {'edge': ('stepfather', 'IsA', 'father'), 'example': None},
 {'edge': ('stepfather', 'DerivedFrom', 'father'), 'example': None}]

In [4]:
get_relations_between_words('bank', 'chase')

[{'edge': ('chase', 'IsA', 'bank'), 'example': None}]

In [5]:
get_relations_between_words('bank', 'bank of china')

[{'edge': ('bank of china', 'IsA', 'bank'), 'example': None}]

In [6]:
get_relations_between_words('bank', 'teller')

[{'edge': ('teller', 'RelatedTo', 'bank'), 'example': None},
 {'edge': ('bank', 'RelatedTo', 'teller'),
  'example': '[[bank]] is related to [[teller]]'}]